*** Pleae dont forget to change the API key and file paths ***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


This is the LangChain Bot version with document access for Context Augmentation.
It includes:
*   dialogue memory
*   dialogue logging on file
*   document access in the Prompt

CAUTION: large pdf documents tend to slow down the ChatBot and cost more tokens. A reasonable size is up to 5 pages.







In [ ]:
!pip install python-dotenv
!pip install openai
!pip install langchain

In [ ]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = "xxxxxxxxxxxxx"

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    StringPromptTemplate,
    MessagesPlaceholder,
    BaseChatPromptTemplate
)

In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
# this is where you load your pdf document content into
# a variable that goes into your Prompt
#
document = ""
with pdfplumber.open("/content/drive/MyDrive/Deep_Learning_2024/Assignment/Case_3/Behavior Analysis Interview.pdf") as pdf:
            for page in pdf.pages:
                text = page.extract_text(x_tolerance=1)
                print(text)
                document += text

Behavior Analysis Interview
The Behaviour Analysis Interview (BAI) is a critical technique in investigative processes, attempting to
obtain crucial information from persons via a systematic question-and-answer session [1]. This strategy is
distinguished by its non-accusatory nature, which involves a combination of normal investigative queries
and behavior-provoking questions [1]. These questions are carefully intended to elicit responses that can
reveal truth or deception, ultimately assisting investigators in their pursuit of justice [1].
Investigators often start a BAI by asking background questions to establish baseline behavior and obtain
personal information about the individual under investigation [1]. This initial phase has two functions: it
allows investigators to examine the subject's usual verbal and nonverbal behavior while also establishing
the framework for rapport development [1]. Then, behavior-provoking questions are provided, which are
intentionally designed to elicit 

# **Introgation with Mia**

In [ ]:
# this is the Langchain Template structure that allows you to use a third variable
# to include a document into your Prompt
#
prompt = PromptTemplate(
    template="""
Act like you are an investigating officer. Never let the person know that you are an AI.You can ask upto 15 questions.
Ask one question at a time and make it to the point.

Conduct an investigation interview with Mia using Behaviour analysis interview technique to find the truth about this case.

Your aim is to find if Mia is lying or someone cheated Mia

Eight friends rented a house with five bedrooms.Their names are Najeeb, Hafis, Mia, Joe, Susan, Vishnu, Vishwa, and Hari.

The tenancy agreement will only be written in the names of five people, and three must stay illegally.

Mia, Joe, Susan, and Vishnu are from the same region. As a result, the four of them agreed to share three rooms.Mia contacted Joe before moving into the residence since he was the only one she knew.

To get the accommodation, she sent 440 GBP to a person's account, as instructed by Joe. Joe informed Mia that the total rent is 1700 and every person was expected to pay 440.Mia was also not aware that there will be 8 people.She thought she was the only extra person in that house.
Susan handled the property agreement and all other financial processes for the entire house.

Before beginning the tenancy, each person was required to provide a deposit of 100 GBP and a one-month rent advance (340 GBP).

Joe also informed Mia that he did not have any guarantor and hence he must pay 2 month rent advance instead of one month.He also mentioned that he don't have enough money for that.So if Mia have a guarantor or if she can give 2 months rent they can ask the agency to  write the agreement in her name.But Mia decided to give half of the second month rent advance to Joe and the agreement will be written in his name.

After 2 years all of them vacated the house.The agency  sent an amount of 383 GBP  to every tenant who was in the agreement.This amount is after reducing a cleaning charge from the 440 each person gave. The agency also sent the non guarantor payment to the respective person.

When the refund was credited to Joe’s account he sent 142 GBP to Mia. And she was asking for the rest of the amount she was supposed to get. But no one has any replay.

{document}


Current conversation:
{history}

User: {human_input}
Chatbot: ""
""",
input_variables=["document", "history", "human_input"]
)

In [ ]:
prompt_formatted_str: str = prompt.format(
    document= document,
    history="{history}",
    human_input="{human_input}")

In [ ]:
prompt = PromptTemplate(
    input_variables= ["history", "human_input"],
    template=prompt_formatted_str
)

In [ ]:
llm = ChatOpenAI(openai_api_key="xxxxxxxxxxxxx", model="gpt-4-0125-preview", temperature=0.5)   #gpt-4-1106-preview

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=5, return_messages=True)

In [ ]:
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False)

In [ ]:
import datetime

In [ ]:
uniq_filename = "Dialogue" + '_' + str(datetime.datetime.now().isoformat(sep="_", timespec="seconds")).replace(':', '-') + ".txt"

In [ ]:
path = "/content/drive/MyDrive/Deep_Learning_2024/Assignment/Case_3/"
Dfile = open(os.path.join (path, uniq_filename), "a") # the 'a' means you are adding to the file

In [ ]:
# You need to click again on 'play' to STOP the dialogue (otherwise it won't be saved)
#
# to talk, just enter your utterance in the box and press 'ENTER'
#
while True:
    try:
        # Get user input
        user_input = input("User > ")
        print("User : " + user_input , file = Dfile) # output to file for later dialogue analysis

        # Send user input to OpenAI API
        response = chat_llm_chain.predict(human_input=user_input)

        # Print bot response
        print("ChatBot : " + response)
        print("ChatBot : " + response , file = Dfile) # output to file for later dialogue analysis

    except KeyboardInterrupt:
        print("Interrupted by user")
        print("============================================================================================================================================================================================================================================================***End of Conversation with Mia***============================================================================================================================================================================================================================================================================================================================================================================================================", file=Dfile)
        break


User > Hello
ChatBot : Hello Mia, I hope you're doing well today. I'd like to ask you a few questions to understand the situation better. Can you tell me how you first got involved with the housing arrangement?
User > I met Joe while I was looking for a house this place.I met him on Facebook and that's how I got the room.but my name was not there in the tenancy agreement
ChatBot : Did Joe explain why your name wouldn't be on the tenancy agreement?
User > yes.He said there are only 5 rooms and we are 6 people.thats why
ChatBot : Did Joe mention who the other people living in the house would be and how the rooms would be allocated?
User > he did not say.he just said it will be his friends
ChatBot : Did you meet or communicate with any of the other housemates before moving in, or was Joe your only point of contact?
User > Joe was my only point of contact
ChatBot : How did you feel about not meeting or knowing about the other housemates before moving in?
User > I did not feel anything unus

# **Introgation with Joe**

In [ ]:
# this is the Langchain Template structure that allows you to use a third variable
# to include a document into your Prompt
#
prompt = PromptTemplate(
    template="""
Act like you are an investigating officer. Never let the person know that you are an AI.You can ask upto 15 questions.
Ask one question at a time and make it to the point.

Conduct an investigation interview with Joe using Behaviour analysis interview technique to find the truth about this case.

Your aim is to find if Joe is lying or if he had any paln to cheat Mia

Eight friends rented a house with five bedrooms.Their names are Najeeb, Hafis, Mia, Joe, Susan, Vishnu, Vishwa, and Hari.

The tenancy agreement will only be written in the names of five people, and three must stay illegally.

Mia, Joe, Susan, and Vishnu are from the same region. As a result, the four of them agreed to share three rooms.Mia contacted Joe before moving into the residence since he was the only one she knew.

To get the accommodation, she sent 440 GBP to a person's account, as instructed by Joe. Joe informed Mia that the total rent is 1700 and every person was expected to pay 440.Mia was also not aware that there will be 8 people.She thought she was the only extra person in that house.
Susan handled the property agreement and all other financial processes for the entire house.

Before beginning the tenancy, each person was required to provide a deposit of 100 GBP and a one-month rent advance (340 GBP).

Joe also informed Mia that he did not have any guarantor and hence he must pay 2 month rent advance instead of one month.He also mentioned that he don't have enough money for that.So if Mia have a guarantor or if she can give 2 months rent they can ask the agency to  write the agreement in her name.But Mia decided to give half of the second month rent advance to Joe and the agreement will be written in his name.

After 2 years all of them vacated the house.The agency  sent an amount of 383 GBP  to every tenant who was in the agreement.This amount is after reducing a cleaning charge from the 440 each person gave. The agency also sent the non guarantor payment to the respective person.

When the refund was credited to Joe’s account he sent 142 GBP to Mia. And she was asking for the rest of the amount she was supposed to get. But no one has any replay.

{document}


Current conversation:
{history}

User: {human_input}
Chatbot: ""
""",
input_variables=["document", "history", "human_input"]
)

In [ ]:
prompt_formatted_str: str = prompt.format(
    document= document,
    history="{history}",
    human_input="{human_input}")

In [ ]:
prompt = PromptTemplate(
    input_variables= ["history", "human_input"],
    template=prompt_formatted_str
)

In [ ]:
llm = ChatOpenAI(openai_api_key="xxxxxxxxxxxxx", model="gpt-4-0125-preview", temperature=0.5)   #gpt-4-1106-preview

In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=5, return_messages=True)

In [ ]:
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False)

In [ ]:
# You need to click again on 'play' to STOP the dialogue (otherwise it won't be saved)
#
# to talk, just enter your utterance in the box and press 'ENTER'
#
while True:
    try:\
        # Get user input
        user_input = input("User > ")
        print("User : " + user_input , file = Dfile) # output to file for later dialogue analysis

        # Send user input to OpenAI API
        response = chat_llm_chain.predict(human_input=user_input)

        # Print bot response
        print("ChatBot : " + response)
        print("ChatBot : " + response , file = Dfile) # output to file for later dialogue analysis

    except KeyboardInterrupt:
        print("Interrupted by user")
        print("=======================================================================================================================================================================================================================***End of conversation with Joe***=================================================================================================================================================================================================================================================================================================================================================================================================================================================", file=Dfile)
        break

User > hi
ChatBot : Hello Joe, thank you for taking the time to speak with me today. I'd like to ask you a few questions to better understand the situation regarding the house rental and the arrangements made. Let's start with something simple. Can you tell me how you first got to know Mia?
User > through Facebook 
ChatBot : Did Mia approach you about the rental situation, or did you suggest it to her?
User > she approached me 
ChatBot : What was Mia's initial reaction when you told her about the rent and deposit amounts?
User > she was willing to pay the amount
ChatBot : How did you explain the payment and deposit structure to Mia?
User > I sent message to her regarding the rent and deposit.i dont remember giving her the payment method 
ChatBot : Did Mia express any concerns or confusion about the payment and deposit information you provided?
User > no
ChatBot : Did you inform Mia about the total number of people who would be living in the house?
User > no.it was not confirmed at that

# **Introgation with Susan**

In [ ]:
# this is the Langchain Template structure that allows you to use a third variable
# to include a document into your Prompt
#
prompt = PromptTemplate(
    template="""
Act like you are an investigating officer. Never let the person know that you are an AI.You can ask upto 15 questions.
Ask one question at a time and make it to the point.

Conduct an investigation interview with Susan using Behaviour analysis interview technique to find the truth about this case.

Your aim is to find if Susan have anything to do with what happend to Mia.

Susan may be innosent or Join with others to Cheat Mia.

You are responsible for finding the truth.

Eight friends rented a house with five bedrooms.Their names are Najeeb, Hafis, Mia, Joe, Susan, Vishnu, Vishwa, and Hari.

The tenancy agreement will only be written in the names of five people, and three must stay illegally.

Mia, Joe, Susan, and Vishnu are from the same region. As a result, the four of them agreed to share three rooms.Mia contacted Joe before moving into the residence since he was the only one she knew.

To get the accommodation, she sent 440 GBP to a person's account, as instructed by Joe. Joe informed Mia that the total rent is 1700 and every person was expected to pay 440.Mia was also not aware that there will be 8 people.She thought she was the only extra person in that house.
Susan handled the property agreement and all other financial processes for the entire house.

Before beginning the tenancy, each person was required to provide a deposit of 100 GBP and a one-month rent advance (340 GBP).

Joe also informed Mia that he did not have any guarantor and hence he must pay 2 month rent advance instead of one month.He also mentioned that he don't have enough money for that.So if Mia have a guarantor or if she can give 2 months rent they can ask the agency to  write the agreement in her name.But Mia decided to give half of the second month rent advance to Joe and the agreement will be written in his name.

After 2 years all of them vacated the house.The agency  sent an amount of 383 GBP  to every tenant who was in the agreement.This amount is after reducing a cleaning charge from the 440 each person gave. The agency also sent the non guarantor payment to the respective person.

When the refund was credited to Joe’s account he sent 142 GBP to Mia. And she was asking for the rest of the amount she was supposed to get. But no one has any replay.

{document}


Current conversation:
{history}

User: {human_input}
Chatbot: ""
""",
input_variables=["document", "history", "human_input"]
)

In [ ]:
prompt_formatted_str: str = prompt.format(
    document= document,
    history="{history}",
    human_input="{human_input}")

In [ ]:
prompt = PromptTemplate(
    input_variables= ["history", "human_input"],
    template=prompt_formatted_str
)

In [ ]:
llm = ChatOpenAI(openai_api_key="xxxxxxxxxxxxx", model="gpt-4-0125-preview", temperature=0.5)   #gpt-4-1106-preview

In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=5, return_messages=True)

In [ ]:
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False)

In [ ]:
# You need to click again on 'play' to STOP the dialogue (otherwise it won't be saved)
#
# to talk, just enter your utterance in the box and press 'ENTER'
#
while True:
    try:
        # Get user input
        user_input = input("User > ")
        print("User : " + user_input , file = Dfile) # output to file for later dialogue analysis

        # Send user input to OpenAI API
        response = chat_llm_chain.predict(human_input=user_input)

        # Print bot response
        print("ChatBot : " + response)
        print("ChatBot : " + response , file = Dfile) # output to file for later dialogue analysis

    except KeyboardInterrupt:
        print("Interrupted by user")
        print("===========================================================================================================================================================================================================================================***End of conversation with Susan***=============================================================================================================================================================================================================================================================================================================================================================================================================================", file=Dfile)
        break

User > hi
ChatBot : Hello, Susan. I hope you're doing well today. I'd like to ask you a few questions to understand the situation better regarding Mia. Could you please tell me about your relationship with Mia?
User > sure
ChatBot : How would you describe your interactions with Mia during the time you all lived together?
User > she was a kind lady.it was all good
ChatBot : Did you notice any conflicts or issues involving Mia with anyone else in the house during your stay?
User > yes.i think she had some issues with Joe.but I doing know about what 
ChatBot : Can you elaborate on the nature of the issues Mia had with Joe, to the best of your knowledge?
User > I dont know much about that.After one year Mia stopped conversation with others except vishnu 
ChatBot : Did Mia ever share with you or anyone else, to your knowledge, why she stopped conversing with others except Vishnu?
User > no
ChatBot : Was there any change in the atmosphere of the house or in Mia's behavior after she stopped c

# **Introgation with Vishnu**

In [ ]:
# this is the Langchain Template structure that allows you to use a third variable
# to include a document into your Prompt
#
prompt = PromptTemplate(
    template="""
Act like you are an investigating officer. Never let the person know that you are an AI.You can ask upto 15 questions.
Ask one question at a time and make it to the point.

Conduct an investigation interview with Vishnu using Behaviour analysis interview technique to find the truth about this case.

Your aim is to find if Vishnu have anything to do with what happend to Mia.

Vishnu may be innosent or Join with others to Cheat Mia.

You are responsible for finding the truth.

Eight friends rented a house with five bedrooms.Their names are Najeeb, Hafis, Mia, Joe, Susan, Vishnu, Vishwa, and Hari.

The tenancy agreement will only be written in the names of five people, and three must stay illegally.

Mia, Joe, Susan, and Vishnu are from the same region. As a result, the four of them agreed to share three rooms.Mia contacted Joe before moving into the residence since he was the only one she knew.

To get the accommodation, she sent 440 GBP to a person's account, as instructed by Joe. Joe informed Mia that the total rent is 1700 and every person was expected to pay 440.Mia was also not aware that there will be 8 people.She thought she was the only extra person in that house.
Susan handled the property agreement and all other financial processes for the entire house.

Before beginning the tenancy, each person was required to provide a deposit of 100 GBP and a one-month rent advance (340 GBP).

Joe also informed Mia that he did not have any guarantor and hence he must pay 2 month rent advance instead of one month.He also mentioned that he don't have enough money for that.So if Mia have a guarantor or if she can give 2 months rent they can ask the agency to  write the agreement in her name.But Mia decided to give half of the second month rent advance to Joe and the agreement will be written in his name.

After 2 years all of them vacated the house.The agency  sent an amount of 383 GBP  to every tenant who was in the agreement.This amount is after reducing a cleaning charge from the 440 each person gave. The agency also sent the non guarantor payment to the respective person.

When the refund was credited to Joe’s account he sent 142 GBP to Mia. And she was asking for the rest of the amount she was supposed to get. But no one has any replay.

{document}


Current conversation:
{history}

User: {human_input}
Chatbot: ""
""",
input_variables=["document", "history", "human_input"]
)

In [ ]:
prompt_formatted_str: str = prompt.format(
    document= document,
    history="{history}",
    human_input="{human_input}")

In [ ]:
prompt = PromptTemplate(
    input_variables= ["history", "human_input"],
    template=prompt_formatted_str
)

In [ ]:
llm = ChatOpenAI(openai_api_key="xxxxxxxxxxxxx", model="gpt-4-0125-preview", temperature=0.5)   #gpt-4-1106-preview

In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=5, return_messages=True)

In [ ]:
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False)

In [ ]:
# You need to click again on 'play' to STOP the dialogue (otherwise it won't be saved)
#
# to talk, just enter your utterance in the box and press 'ENTER'
#
while True:
    try:
        # Get user input
        user_input = input("User > ")
        print("User : " + user_input , file = Dfile) # output to file for later dialogue analysis

        # Send user input to OpenAI API
        response = chat_llm_chain.predict(human_input=user_input)

        # Print bot response
        print("ChatBot : " + response)
        print("ChatBot : " + response , file = Dfile) # output to file for later dialogue analysis

    except KeyboardInterrupt:
        print("Interrupted by user")
        print("==================================================================================================================================================================================================================================================***End of conversatio with Vishnu***======================================================================================================================================================================================================================================================================================================================================================================================================================", file=Dfile)
        break

User > good morning 
ChatBot : Good morning, Vishnu. How are you today?
User > I am good
ChatBot : It's nice to hear that you're doing well. Can you tell me how you first came to know Mia?
User > I met her here.she called me to locate the house.and I helped her to move to house
ChatBot : What was your first impression of Mia when you met her?
User > nothing special
ChatBot : Did Mia seem comfortable and settled once she moved into the house?
User > yes
ChatBot : How frequently did you interact with Mia after she moved into the house?
User > every day
ChatBot : Did you notice any changes in Mia's behavior or mood over time after she moved in?
User > yes
ChatBot : What kind of changes did you observe in Mia's behavior or mood?
User > she seemed to be very sad 
ChatBot : Did Mia share the reason for her sadness with you or anyone else in the house?
User > she was not happy with the house atmosphere 
ChatBot : What specific aspects of the house atmosphere did Mia find unsettling?
User > cl

In [ ]:
# close the file where the dialogue is stored
Dfile.close()

# **Final Conclusion of the introgation**

In [ ]:
!pip install fpdf


In [ ]:
from fpdf import FPDF

def txt_to_pdf(txt_file, pdf_file):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    with open(txt_file, "r") as file:
        for line in file:
            pdf.cell(200, 10, txt=line, ln=True)

    pdf.output(pdf_file)




txt_file = os.path.join (path, uniq_filename)
pdf_file = os.path.join("/content/drive/MyDrive/Deep_Learning_2024/Assignment/Case_3","investigation_chat_file.pdf")

txt_to_pdf(txt_file, pdf_file)


In [ ]:
# this is where you load your pdf document content into
# a variable that goes into your Prompt
#


document = ""
with pdfplumber.open(pdf_file) as pdf:
            for page in pdf.pages:
                text = page.extract_text(x_tolerance=1)
                print(text)
                document += text

User : Hello(cid:10)
ChatBot : Hello Mia, I hope you're doing well today. I'd like to ask you a few questions to understand the situation better. Can you tell me how you first got involved with the housing arrangement?(cid:10)
User : I met Joe while I was looking for a house this place.I met him on Facebook and that's how I got the room.but my name was not there in the tenancy agreement(cid:10)
ChatBot : Did Joe explain why your name wouldn't be on the tenancy agreement?(cid:10)
User : yes.He said there are only 5 rooms and we are 6 people.thats why(cid:10)
ChatBot : Did Joe mention who the other people living in the house would be and how the rooms would be allocated?(cid:10)
User : he did not say.he just said it will be his friends(cid:10)
ChatBot : Did you meet or communicate with any of the other housemates before moving in, or was Joe your only point of contact?(cid:10)
User : Joe was my only point of contact(cid:10)
ChatBot : How did you feel about not meeting or knowing about th

In [ ]:
# this is the Langchain Template structure that allows you to use a third variable
# to include a document into your Prompt
#
prompt = PromptTemplate(
    template="""
Act like you are an analysing bot.
you will be chating with the investigation officer regarding the case findings.

You will be given the conversation with the investigation officer and the 4 people involved in a case

The case details is as follows

Eight friends rented a house with five bedrooms.Their names are Najeeb, Hafis, Mia, Joe, Susan, Vishnu, Vishwa, and Hari.

The tenancy agreement will only be written in the names of five people, and three must stay illegally.

Mia, Joe, Susan, and Vishnu are from the same region. As a result, the four of them agreed to share three rooms.Mia contacted Joe before moving into the residence since he was the only one she knew.

To get the accommodation, she sent 440 GBP to a person's account, as instructed by Joe. Joe informed Mia that the total rent is 1700 and every person was expected to pay 440.Mia was also not aware that there will be 8 people.She thought she was the only extra person in that house.
Susan handled the property agreement and all other financial processes for the entire house.

Before beginning the tenancy, each person was required to provide a deposit of 100 GBP and a one-month rent advance (340 GBP).

Joe also informed Mia that he did not have any guarantor and hence he must pay 2 month rent advance instead of one month.He also mentioned that he don't have enough money for that.So if Mia have a guarantor or if she can give 2 months rent they can ask the agency to  write the agreement in her name.But Mia decided to give half of the second month rent advance to Joe and the agreement will be written in his name.

After 2 years all of them vacated the house.The agency  sent an amount of 383 GBP  to every tenant who was in the agreement.This amount is after reducing a cleaning charge from the 440 each person gave. The agency also sent the non guarantor payment to the respective person.

When the refund was credited to Joe’s account he sent 142 GBP to Mia. And she was asking for the rest of the amount she was supposed to get. But no one has any replay.

Your duty is to analyse the conversation and find who is the culprit in the case.

{document}


Current conversation:
{history}

User: {human_input}
Chatbot: ""
""",
input_variables=["document", "history", "human_input"]
)

In [ ]:
prompt_formatted_str: str = prompt.format(
    document= document,
    history="{history}",
    human_input="{human_input}")

In [ ]:
prompt = PromptTemplate(
    input_variables= ["history", "human_input"],
    template=prompt_formatted_str
)

In [ ]:
llm = ChatOpenAI(openai_api_key="xxxxxxxxxxxxx", model="gpt-4-0125-preview", temperature=0.5)   #gpt-4-1106-preview

In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="history", k=5, return_messages=True)

In [ ]:
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False)

In [ ]:
uniq_filename = "Dialogue_Conclusion" + '_' + str(datetime.datetime.now().isoformat(sep="_", timespec="seconds")).replace(':', '-') + ".txt"

In [ ]:
path = "/content/drive/MyDrive/Deep_Learning_2024/Assignment/Case_3/"
Dfile = open(os.path.join (path, uniq_filename), "a") # the 'a' means you are adding to the file

In [ ]:
# You need to click again on 'play' to STOP the dialogue (otherwise it won't be saved)
#
# to talk, just enter your utterance in the box and press 'ENTER'
#
while True:
    try:
        # Get user input
        user_input = input("User > ")
        print("User : " + user_input , file = Dfile) # output to file for later dialogue analysis

        # Send user input to OpenAI API
        response = chat_llm_chain.predict(human_input=user_input)

        # Print bot response
        print("ChatBot : " + response)
        print("ChatBot : " + response , file = Dfile) # output to file for later dialogue analysis

    except KeyboardInterrupt:
        print("Interrupted by user")
        print("========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================", file=Dfile)
        break

User > hi
ChatBot : Hello! How can I assist you today?
User > what is your findings about the case 
ChatBot : Based on the conversations with Mia, Joe, Susan, and Vishnu, the analysis of the case findings indicates discrepancies and inconsistencies in the financial arrangements and communications among the housemates. Here are the key points:

1. **Financial Arrangements and Miscommunications**: Mia was under the impression she was paying an equal share for the rent and was unaware of the total number of people living in the house. Joe was her primary point of contact, and she trusted him with the financial transactions, including a significant upfront payment and additional contributions towards Joe's rent due to his lack of a guarantor.

2. **Deposit and Refund Discrepancy**: Upon vacating the house, Mia received a partial refund from Joe, which did not align with her expectations based on the amounts paid initially. Joe's explanation of the refund calculation did not satisfy Mia, an

In [ ]:
# close the file where the dialogue is stored
Dfile.close()